In [ ]:
import os

from matplotlib import ticker
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d.art3d import pathpatch_2d_to_3d, Poly3DCollection
import numpy as np
try:
    import open3d as o3d
except ImportError:
    import sys
    print(sys.exc_info())
import pandas as pd

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.viz import (set_colorblind, fig_config, set_axes_equal,
                              save_fig)
from helpers import (clean_df, export_pcd, export_fields, poynting_vector,
                     export_rect_idx, get_imcolors)

In [ ]:
set_colorblind()
%config InlineBackend.figure_format = 'retina'

# Parametrized averaging surface projection onto the model

In [ ]:
# input data

polarization = 'te'
f = 26
edge_length = 20
origin = [-25, 15]

In [ ]:
# load surface data

df = load_ear_data(polarization, f, surface='front')
df = clean_df(df)
xyz = export_pcd(df)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()

In [ ]:
# extract points visible from the plane wave incidence POV (-x direction)

diameter = np.linalg.norm(
    pcd.get_max_bound() - pcd.get_min_bound()
)
radius = 10 ** 5
camera = [center[0] + diameter, center[1], center[2]]
_, mask = pcd.hidden_point_removal(camera, radius)
xyz_zy = xyz[mask]
rect_center = [origin[0] + edge_length / 2, origin[1] + edge_length / 2]
_, rect_idx = export_rect_idx(xyz_zy, rect_center, edge_length, view='zy')

In [ ]:
skip = 1
scaler = 2
fig_config(latex=True, text_size=20, scaler=scaler)
fig = plt.figure()
ax = plt.axes(projection ='3d')

# point cloud
ax.scatter(xyz[::skip, 2], xyz[::skip, 0], xyz[::skip, 1],
           s=0.25, alpha=0.02, color='gray')
ax.scatter(xyz_zy[rect_idx, 2], xyz_zy[rect_idx, 0], xyz_zy[rect_idx, 1],
           s=0.25, alpha=0.2, color='red')
# handle for legend
rect_handle = Rectangle([0,0], 0, 0, color='red', ec='None', alpha=0.8,
                        label='averaging\nsurface')

# averaging surface(s)
rect_pos = xyz[:, 0].max()
edge_length = 20
param_rect = Rectangle(origin, edge_length, edge_length,
                       ec='black', fc='None', lw=2,
                       label='parametric\nsurface')
ax.add_patch(param_rect)
pathpatch_2d_to_3d(param_rect, z=100, zdir='y')

# additional figure configuration
ax.set(xlabel='$z$ [mm]', ylabel='$x$ [mm]',
       xticks=[10, -10, -30],
       yticks=[50, 70, 90],
       zticks=[0, 20, 40])
ax.zaxis.set_rotate_label(False)
ax.set_zlabel('$y$ [mm]', rotation=90)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 15
ax.yaxis.labelpad = 12
ax.zaxis.labelpad = 12
ax.view_init(15, 70)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
fig.legend(handles=[rect_handle, param_rect],
           bbox_to_anchor=(0.75, 1),
           handlelength=1, handleheight=1,
           ncol=2)
fig.tight_layout();

# save figure
#fname = os.path.join('figures', 'projection_a')
#save_fig(fig, fname=fname, formats=['png'])

In [ ]:
skip = 1
scaler = 2
fig_config(latex=True, text_size=20, scaler=scaler)
fig = plt.figure()
ax = plt.axes(projection ='3d')

# point cloud
ax.scatter(xyz[::skip, 2], xyz[::skip, 0], xyz[::skip, 1],
           s=0.25, alpha=0.02, color='gray')
ax.scatter(xyz_zy[rect_idx, 2], xyz_zy[rect_idx, 0], xyz_zy[rect_idx, 1],
           s=0.25, alpha=0.15, color='red')

# averaging surface(s)
rect_pos = xyz[:, 0].max()
origin = [-25, 15]
edge_length = 20
param_rect = Rectangle(origin, edge_length, edge_length,
                       ec='black', fc='None', lw=2,
                       label='parametrized\nsurface')
ax.add_patch(param_rect)
pathpatch_2d_to_3d(param_rect, z=100, zdir='y')

# additional figure configuration
ax.set(xlabel='$z$ [mm]',
       xticks=[10, -10, -30],
       yticks=[],
       zticks=[0, 20, 40])
ax.zaxis.set_rotate_label(False)
ax.set_zlabel('$y$ [mm]', rotation=90)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 15
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 20
ax.tick_params(axis='z', pad=10)
ax.view_init(0, 90)
ax.set_box_aspect([1, 1, 1])
ax.set_proj_type('ortho')
ax = set_axes_equal(ax)
fig.tight_layout();

# save figure
#fname = os.path.join('figures', 'projection_b')
#save_fig(fig, fname=fname, formats=['png'])